<a href="https://colab.research.google.com/github/romqn1999/Predict-price-after-pattern/blob/main/Predict_price_after_pattern.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
from google.colab import files
uploaded = files.upload()

Saving AAPL_2000-01-01 00_00_00_2021-05-17_30_data_patterns.csv to AAPL_2000-01-01 00_00_00_2021-05-17_30_data_patterns (1).csv
Saving GOOG_2000-01-01 00_00_00_2021-05-17_30_data_patterns.csv to GOOG_2000-01-01 00_00_00_2021-05-17_30_data_patterns.csv
Saving SNP_2000-01-01 00_00_00_2021-05-17_30_data_patterns.csv to SNP_2000-01-01 00_00_00_2021-05-17_30_data_patterns (4).csv


## Import packages

In [42]:
import math
import matplotlib.pyplot as plt
import keras
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from keras import optimizers
from keras.optimizers import Adagrad, Adadelta, RMSprop, Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping

## Get data at pattern for training

In [43]:
import io
import pandas as pd

price_patterns_df = pd.DataFrame()
for filename in uploaded.keys():
    print(filename)
    if 'data_patterns.csv' not in filename:
        print('Skipping file', filename)
        continue
    df = pd.read_csv(io.StringIO(uploaded[filename].decode('utf-8')),
                     header=None)
    price_patterns_df = price_patterns_df.append(df)

price_patterns_df

AAPL_2000-01-01 00_00_00_2021-05-17_30_data_patterns.csv
GOOG_2000-01-01 00_00_00_2021-05-17_30_data_patterns.csv
SNP_2000-01-01 00_00_00_2021-05-17_30_data_patterns.csv


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30
0,5.032500,4.680357,4.530000,4.596786,4.711786,4.580000,3.759286,4.059286,3.897143,3.575000,3.466786,3.505000,3.184286,3.206786,3.169286,3.457143,3.937857,3.717143,3.498214,3.638929,3.478571,3.515714,3.267500,3.459643,3.508214,3.442143,3.288929,3.568214,3.733929,3.965714,3.842500
1,3.575000,3.466786,3.505000,3.184286,3.206786,3.169286,3.457143,3.937857,3.717143,3.498214,3.638929,3.478571,3.515714,3.267500,3.459643,3.508214,3.442143,3.288929,3.568214,3.733929,3.965714,3.842500,3.820000,3.963929,3.689286,3.539286,3.508571,3.424286,3.384643,3.218571,3.444286
2,3.064643,3.093214,3.081786,3.048214,3.241071,3.377857,3.322143,3.250357,3.310714,3.235000,3.166429,3.132500,3.047500,2.977857,2.940357,2.792857,2.958214,3.155714,3.155714,3.201429,3.240357,3.364286,3.321429,3.218929,3.268214,3.320714,3.341071,3.445000,3.561429,3.661071,3.493929
3,3.064643,3.093214,3.081786,3.048214,3.241071,3.377857,3.322143,3.250357,3.310714,3.235000,3.166429,3.132500,3.047500,2.977857,2.940357,2.792857,2.958214,3.155714,3.155714,3.201429,3.240357,3.364286,3.321429,3.218929,3.268214,3.320714,3.341071,3.445000,3.561429,3.661071,3.493929
4,3.064643,3.093214,3.081786,3.048214,3.241071,3.377857,3.322143,3.250357,3.310714,3.235000,3.166429,3.132500,3.047500,2.977857,2.940357,2.792857,2.958214,3.155714,3.155714,3.201429,3.240357,3.364286,3.321429,3.218929,3.268214,3.320714,3.341071,3.445000,3.561429,3.661071,3.493929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,45.930000,45.889999,46.200001,44.560001,45.080002,45.880001,45.049999,44.430000,43.599998,43.959999,44.540001,44.259998,43.799999,43.750000,43.480000,43.619999,43.480000,43.500000,42.959999,42.279999,41.540001,40.470001,40.000000,40.389999,40.549999,40.240002,40.459999,40.310001,40.029999,40.810001,40.849998
148,44.430000,43.599998,43.959999,44.540001,44.259998,43.799999,43.750000,43.480000,43.619999,43.480000,43.500000,42.959999,42.279999,41.540001,40.470001,40.000000,40.389999,40.549999,40.240002,40.459999,40.310001,40.029999,40.810001,40.849998,41.049999,41.389999,40.880001,41.320000,41.459999,39.169998,38.970001
149,45.570000,46.869999,47.040001,46.230000,46.049999,45.570000,46.410000,46.720001,47.490002,48.099998,45.080002,45.639999,47.180000,46.560001,46.139999,45.910000,43.939999,43.130001,43.349998,44.259998,44.099998,44.270000,44.389999,44.580002,44.910000,43.820000,43.310001,43.700001,44.439999,45.160000,45.049999
150,57.490002,57.119999,55.020000,54.919998,53.779999,55.599998,55.880001,57.200001,57.410000,56.349998,55.669998,56.080002,56.090000,57.650002,57.349998,56.410000,54.470001,53.290001,52.790001,51.959999,52.270000,51.560001,53.080002,53.759998,53.490002,52.730000,53.720001,52.840000,52.919998,52.430000,52.669998


In [44]:
#@title Separate into input and output columns: Get data: X and y

X_df = price_patterns_df[price_patterns_df.columns[:-1]]
y_df = price_patterns_df[price_patterns_df.columns[-1:]]
print(X_df)
print(y_df)

X_data = X_df.values
y_data = y_df.values
print(X_data)
print(y_data)

            0          1          2   ...         27         28         29
0     5.032500   4.680357   4.530000  ...   3.568214   3.733929   3.965714
1     3.575000   3.466786   3.505000  ...   3.424286   3.384643   3.218571
2     3.064643   3.093214   3.081786  ...   3.445000   3.561429   3.661071
3     3.064643   3.093214   3.081786  ...   3.445000   3.561429   3.661071
4     3.064643   3.093214   3.081786  ...   3.445000   3.561429   3.661071
..         ...        ...        ...  ...        ...        ...        ...
147  45.930000  45.889999  46.200001  ...  40.310001  40.029999  40.810001
148  44.430000  43.599998  43.959999  ...  41.320000  41.459999  39.169998
149  45.570000  46.869999  47.040001  ...  43.700001  44.439999  45.160000
150  57.490002  57.119999  55.020000  ...  52.840000  52.919998  52.430000
151  56.349998  55.669998  56.080002  ...  54.709999  54.880001  53.330002

[304 rows x 30 columns]
            30
0     3.842500
1     3.444286
2     3.493929
3     3.493929


In [45]:
#@title Preprocess & split data for train model
# Train, test split
train_ratio = 0.8 #@param {type:"number"}
if train_ratio < 0 or  train_ratio > 1:
    train_ratio = 0.8
n_data = len(X_data)
n_train = int(train_ratio * n_data)
n_test = n_data - n_train
# Feature Scaling
scale_method = "MinMax" #@param ["None", "MinMax"]

X_train, y_train = X_data[: n_train], y_data[: n_train]
X_test, y_test = X_data[n_train:], y_data[n_train:]
if scale_method == 'MinMax':
    sc_train = MinMaxScaler(feature_range = (0, 1))
    X_train_scaled = sc_train.fit_transform(X_train.T).T
    y_train_scaled = sc_train.transform(y_train.T).T
    sc_test = MinMaxScaler(feature_range = (0, 1))
    X_test_scaled = sc_test.fit_transform(X_test.T).T
    y_test_scaled = sc_test.transform(y_test.T).T
    # X_train, y_train = X_train_scaled, y_train_scaled
    # X_test, y_test = X_test_scaled, y_test_scaled
print(X_train.shape, y_train.shape)
print(X_train_scaled.shape, y_train_scaled.shape)
print(X_test.shape, y_test.shape)
print(X_test_scaled.shape, y_test_scaled.shape)

# print(X_train)
# print(y_train)

(243, 30) (243, 1)
(243, 30) (243, 1)
(61, 30) (61, 1)
(61, 30) (61, 1)


## LSTM model

### Build model

In [46]:
#@title Build & fit model
lstm_units = 50 #@param {type:"integer"}
dropout_prob = 0.5 #@param {type:"number"}
epochs = 8000 #@param {type:"integer"}
batch_size = 512 #@param {type:"integer"}

model = Sequential()
#Adding the first LSTM layer and some Dropout regularisation
model.add(LSTM(units=lstm_units,
               return_sequences=True,
               input_shape=(X_data.shape[1], 1)))
model.add(Dropout(dropout_prob))
# Adding a second LSTM layer and some Dropout regularisation
model.add(LSTM(units=lstm_units,
               return_sequences=True))
model.add(Dropout(dropout_prob))
# Adding a third LSTM layer and some Dropout regularisation
model.add(LSTM(units=lstm_units,
               return_sequences=True))
model.add(Dropout(dropout_prob))
# Adding a fourth LSTM layer and some Dropout regularisation
model.add(LSTM(units=lstm_units))
model.add(Dropout(dropout_prob))
# Adding the output layer
model.add(Dense(units=1))

# Adam optimizer
opt = Adam(learning_rate=0.0005, epsilon=1e-06, decay=1e-06)
# # PiecewiseConstantDecay optimizer
# boundaries = [200, 800, 2000, 4000]
# values = [0.01, 0.001, 0.0005, 0.0001, 0.00001]
# opt = keras.optimizers.schedules.PiecewiseConstantDecay(
#     boundaries, values)
# Compiling the RNN
# model.compile(optimizer = opt, loss = 'mean_squared_error')
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

### Train model

In [ ]:
def train_lstm(model, X_train, y_train, epochs, batch_size):
    # We have now reshaped the data into the following format (#values, #time-steps, #1 dimensional output).
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    y_train = np.reshape(y_train, (y_train.shape[0], 1))

    # Fitting the RNN to the Training set
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)
    return model

model = train_lstm(model, X_train_scaled, y_train_scaled, epochs, batch_size)

Epoch 1/8000
1/1 [==============================] - 6s 6s/step - loss: 0.4691
Epoch 2/8000
1/1 [==============================] - 0s 188ms/step - loss: 0.3959
Epoch 3/8000
1/1 [==============================] - 0s 179ms/step - loss: 0.3290
Epoch 4/8000
1/1 [==============================] - 0s 187ms/step - loss: 0.2658
Epoch 5/8000
1/1 [==============================] - 0s 189ms/step - loss: 0.2217
Epoch 6/8000
1/1 [==============================] - 0s 193ms/step - loss: 0.2048
Epoch 7/8000
1/1 [==============================] - 0s 198ms/step - loss: 0.2342
Epoch 8/8000
1/1 [==============================] - 0s 199ms/step - loss: 0.2258
Epoch 9/8000
1/1 [==============================] - 0s 189ms/step - loss: 0.2105
Epoch 10/8000
1/1 [==============================] - 0s 194ms/step - loss: 0.1885
Epoch 11/8000
1/1 [==============================] - 0s 184ms/step - loss: 0.1874
Epoch 12/8000
1/1 [==============================] - 0s 199ms/step - loss: 0.1933
Epoch 13/8000
1/1 [=========

### Test model

In [ ]:
# X_test = X_train

In [ ]:
predicted_stock_price = model.predict(np.reshape(X_test_scaled, (X_test_scaled.shape[0], X_test_scaled.shape[1], 1)))
if scale_method == 'MinMax':
    predicted_stock_price = sc_test.inverse_transform(predicted_stock_price.T).T
    # y_test = sc_test.inverse_transform(y_test.T).T

In [ ]:
y_test, predicted_stock_price

In [ ]:
float_formatter = "{:.2f}".format
np.set_printoptions(formatter={'float_kind':float_formatter})
print(y_test - predicted_stock_price)
print(max(abs(y_test - predicted_stock_price)))

In [ ]:
# Calculate RMSE and MAPE
from sklearn.metrics import mean_squared_error

def get_mape(y_true, y_pred): 
    """
    Compute mean absolute percentage error (MAPE)
    """
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

rmse = math.sqrt(mean_squared_error(y_test, predicted_stock_price))
mape = get_mape(y_test, predicted_stock_price)

rmse, mape

In [ ]:
# Visualising the results
plt.figure(figsize=(20, 10), dpi= 120, facecolor='w', edgecolor='k')
plt.plot(range(len(y_test)), y_test, 'o', color = 'red', label = 'y test groundtruth')
plt.plot(range(len(y_test)), predicted_stock_price, 'o', color = 'blue', label = 'Predicted Price')
plt.vlines(range(len(y_test)), np.minimum(y_test, predicted_stock_price), np.maximum(y_test, predicted_stock_price))
# plt.xticks(np.arange(0,459,50))
plt.title('LSTM Price Prediction')
plt.xlabel('Sample id')
plt.ylabel('Stock Price')
plt.legend()
plt.show()

## XGBoost model

### Build model

In [ ]:
#@title Build model
seed = 100 #@param {type:"integer"}
n_estimators = 100 #@param {type:"integer"}
max_depth = 3 #@param {type:"integer"}
learning_rate = 0.1 #@param {type:"number"}
min_child_weight = 0 #@param {type:"number"}
subsample = 1 #@param {type:"number"}
colsample_bytree = 1 #@param {type:"number"}
colsample_bylevel = 1 #@param {type:"number"}
gamma = 0 #@param {type:"number"}
from xgboost import XGBRegressor

xgbmodel = XGBRegressor(seed=seed,
                        n_estimators=n_estimators,
                        max_depth=max_depth,
                        learning_rate=learning_rate,
                        min_child_weight=min_child_weight,
                        subsample=subsample,
                        colsample_bytree=colsample_bytree,
                        colsample_bylevel=colsample_bylevel,
                        gamma=gamma)

### Train the model

In [ ]:
xgbmodel.fit(np.reshape(X_train_scaled, (X_train_scaled.shape[0], X_train_scaled.shape[1])), np.reshape(y_train_scaled, (y_train_scaled.shape[0])))

### Test model

In [ ]:
predicted_stock_price = xgbmodel.predict(np.reshape(X_test_scaled, (X_test_scaled.shape[0], X_test_scaled.shape[1])))
predicted_stock_price = np.reshape(predicted_stock_price, (predicted_stock_price.shape[0], 1))
if scale_method == 'MinMax':
    predicted_stock_price = sc_test.inverse_transform(predicted_stock_price.T).T
    # y_test = sc_test.inverse_transform(y_test.T).T

In [ ]:
float_formatter = "{:.2f}".format
np.set_printoptions(formatter={'float_kind':float_formatter})
print(y_test - predicted_stock_price)
print(max(abs(y_test - predicted_stock_price)))

In [ ]:
rmse = math.sqrt(mean_squared_error(y_test, predicted_stock_price))
mape = get_mape(y_test, predicted_stock_price)

rmse, mape

In [ ]:
# Visualising the results
plt.figure(figsize=(20, 10), dpi= 120, facecolor='w', edgecolor='k')
plt.plot(range(len(y_test)), y_test, 'o', color = 'red', label = 'y test groundtruth')
plt.plot(range(len(y_test)), predicted_stock_price, 'o', color = 'blue', label = 'Predicted Price')
plt.vlines(range(len(y_test)), np.minimum(y_test, predicted_stock_price), np.maximum(y_test, predicted_stock_price))
# plt.xticks(np.arange(0,459,50))
plt.title('XGBoost Price Prediction')
plt.xlabel('Sample id')
plt.ylabel('Stock Price')
plt.legend()
plt.show()